### Header for selecting leftout compound directory to work in

In [17]:
loo_pert_id_idx = 1


loo_pert_id_idx =

     1



In [18]:
unique_kabx_cmpds_tbl = rtable('/idi/cgtb/abond/combine_screens_456/pcls_for_manuscript_final/loocv_pcls/kabx_pert_ids_tbl_for_loocv.txt');

size(unique_kabx_cmpds_tbl)
headt(unique_kabx_cmpds_tbl)

Reading /idi/cgtb/abond/combine_screens_456/pcls_for_manuscript/loocv_pcls/kabx_pert_ids_tbl_for_loocv.txt


ans =

   434     2


ans =

  2x3 table

    idx          field                value      
    ___    _________________    _________________

     1     {'kabx_cmpd_idx'}    {[            1]}
     2     {'kabx_cmpd'    }    {'BRD-A02179977'}



In [19]:
this_loocv_cmpd = unique_kabx_cmpds_tbl.kabx_cmpd(unique_kabx_cmpds_tbl.kabx_cmpd_idx == loo_pert_id_idx)

this_loocv_cmpd_string = strjoin(this_loocv_cmpd)


this_loocv_cmpd =

  1x1 cell array

    {'BRD-A02179977'}


this_loocv_cmpd_string =

    'BRD-A02179977'



### End header

In [20]:
datadir = '/idi/cgtb/morzech/idmp/combine_screens_456';
wkdir = ['/idi/cgtb/abond/combine_screens_456/pcls_for_manuscript_final/loocv_pcls/', 'leave_out_cmpd_', this_loocv_cmpd_string]
%wkdir = fullfile(datadir,'pcls_remove_gsk_no_broad_sar_pearson');

mk_cd_dir(wkdir,true)
imatlab_export_fig('print-png')


wkdir =

    '/idi/cgtb/abond/combine_screens_456/pcls_for_manuscript/loocv_pcls/leave_out_cmpd_BRD-A02179977'



In [21]:
g = glob(fullfile(wkdir,'grzs_for_pcls_screens456_pearson_corr_n*.gctx'));
grzs_corr = parse_gctx(g{1});

g = glob(fullfile(wkdir,'grzs_for_pcls_screens456_pearson_corr_rank_n*.gctx'));
grzs_corr_rank = parse_gctx(g{1});

Reading /idi/cgtb/abond/combine_screens_456/pcls_for_manuscript/loocv_pcls/leave_out_cmpd_BRD-A02179977/grzs_for_pcls_screens456_pearson_corr_n9357x9357.gctx [9357x9357]
Done [7.12 s].
Reading /idi/cgtb/abond/combine_screens_456/pcls_for_manuscript/loocv_pcls/leave_out_cmpd_BRD-A02179977/grzs_for_pcls_screens456_pearson_corr_rank_n9357x9357.gctx [9357x9357]
Done [7.07 s].


In [ ]:
addpath('/idi/cgtb/abond/combine_screens_456/pcls_for_manuscript_final/loocv_pcls')

In [20]:
pcls = parse_gmt(fullfile(wkdir,'new_pcls.gmt'));

In [21]:
run_pcl_concordance_analysis(wkdir, grzs_corr, grzs_corr_rank, pcls, true, 'verify_pcls/figures', true, 'verify_pcls/gcts')

Parse gctx with correlation
Parse gctx with correlation ranks
Remove classes with less than 3 members
Run summary and make figures
1/70
Similarity matrix detected: Transforming to distances
Saving file to /idi/cgtb/abond/combine_screens_456/pcls_for_manuscript/verify_pcls/gcts/30S_ribosome_corr_n477x477.gct
Dimensions of matrix: [477x477]
Setting precision to 4
Saved.

ans =

    '/idi/cgtb/abond/combine_screens_456/pcls_for_manuscript/verify_pcls/gcts/30S_ribosome_corr_n477x477.gct'

Saving file to /idi/cgtb/abond/combine_screens_456/pcls_for_manuscript/verify_pcls/gcts/30S_ribosome_corr_rank_n477x477.gct
Dimensions of matrix: [477x477]
Setting precision to 4
Saved.

ans =

    '/idi/cgtb/abond/combine_screens_456/pcls_for_manuscript/verify_pcls/gcts/30S_ribosome_corr_rank_n477x477.gct'

Similarity matrix detected: Transforming to distances
Saving file to /idi/cgtb/abond/combine_screens_456/pcls_for_manuscript/verify_pcls/gcts/InfB_corr_n40x40.gct
Dimensions of matrix: [40x40]
Setting

In [22]:
disp('Done')

Done
